In [ ]:
import pandas as pd
from COMETH import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
import csv
import os

gui = nimble.NimbleGUI()
gui.serve(8080)
# sudo lsof -iTCP:8080 -sTCP:LISTEN

s12 = Skeleton('COMETH/BODY12.xml')
print("BODY12 Loaded")
s15 = Skeleton('COMETH/BODY15.xml')
print("BODY15 Loaded")
# Rajagopal
# s = DynamicSkeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml')
# BSM
s = DynamicSkeleton(config='COMETH/BODY15_constrained_3D.xml', \
    # osim_file='/media/rmhri/mrom/data/skel_models_v1.1/bsm.osim',\
    # max_velocity=10
    )

s.qdot_l *= 2
s.qdot_u *= 2

In [ ]:
# Initialize the model
n_subjects = 5
subject_labels = ['subject0', 'subject1', 'subject2', 'subject3', 'subject4']
healthy_arm  = ['R', 'L', 'L', 'L', 'R']

header = [  'pelvis_list', 'pelvis_tilt', 'pelvis_rotation', \
            'pelvis_translation_x', 'pelvis_translation_y', 'pelvis_translation_z', \
            'hip_r_flexion_r', 'hip_r_adduction_r', 'hip_r_rotation_r', \
            'knee_r_flexion', 'ankle_r_flexion', 'subtalar_r', 'mtp_r', \
            'hip_l_flexion_l', 'hip_l_adduction_l', 'hip_l_rotation_l', \
            'knee_l_flexion', 'ankle_l_flexion', 'subtalar_l', 'mtp_l', \
            'lumbar_bending', 'lumbar_extension', 'lumbar_twist', \
            'thorax_bending', 'thorax_extension', 'thorax_twist', \
            'neck_bending', 'neck_extension', 'neck_twist', \
            'scapula_r_abduction', 'scapula_r_elevation', 'scapula_r_upward_rot', \
            'shoulder_r_adduction', 'shoulder_r_flexion', 'shoulder_r_rotation', \
            'elbow_r_flexion', 'radioulnar_r_pro_sup', 'wrist_r_flexion', 'wrist_r_deviation', \
            'scapula_l_abduction', 'scapula_l_elevation', 'scapula_l_upward_rot', \
            'shoulder_l_adduction', 'shoulder_l_flexion', 'shoulder_l_rotation', \
            'elbow_l_flexion', 'radioulnar_l_pro_sup', 'wrist_l_flexion', 'wrist_l_deviation']

# Read the file in Shafagh format and collapse the header into one line
def modify_csv_files(subj):
    side = healthy_arm[subj]
    init_folder = '../data/raw_exports/'
    mod_folder = '../data/COMETH_modified_exports/'
    init_files = [file for file in os.listdir(init_folder) if f'subject{subj}_{side}h' in file and file.endswith('.csv')]    

    for filename in init_files:
        file = str(os.path.join(init_folder, filename))
        print('Reading file:', file)
        out_header = ['']*465
        out_file = []
        with open(file, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for i,row in enumerate(spamreader):
                # if i<7:
                #     print(i, row[10])
                if i > 1 and i < 7 and i != 4:
                    for j in range(len(row)):
                    # for j in range(465):
                        if row[j] == "''" or out_header[j] == '':
                            out_header[j] += row[j]
                        else:
                            out_header[j] += "." + row[j]
                if i > 6:
                    out_file.append(row) 
        
        modified_name = str(os.path.join(mod_folder, filename.replace(".csv",".oneheader.csv")))
        with open(modified_name, 'w',) as f:
            writer = csv.writer(f,delimiter=',')
            writer.writerow(out_header)
            writer.writerows(out_file)

def extract_COMETH_angles(subj):
    
    # kps_optitrack = ['LHand','LFArm','RHand','LUArm','RFArm','RUArm']
    # self.kps =  ['RKnee', 'LWrist', 'RHip', 'RShoulder',  'LElbow', 'LHip', 'RElbow', 'RWrist', 'LKnee', 'LShoulder', 'RAnkle', 'LAnkle']
    # kps =  ['', 'LHand', 'RHip', 'RUArm',  'LFArm', 'LHip', 'RFArm', 'RHand', '', 'LUArm', '', '']
    
    # self.kps = ['Root', 'MidHip', 'MidShoulder', 'RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', \
                                #  'RHip', 'RKee', 'RAnkle' 'LHip', 'LKnee', 'LAnkle']
    kps = ['Skeleton', '', 'Midshoulder', 'RUArm', 'RFArm', 'RHand', 'LUArm', 'LFArm', 'LHand', '', '', '', '', '', '']
    # Read the modified version of the csv in pandas
    side = healthy_arm[subj]
    mod_folder = '../data/COMETH_modified_exports/'
    mod_files = [file for file in os.listdir(mod_folder) if f'subject{subj}_{side}h' in file and file.endswith('.oneheader.csv')]        
    for i, filename in enumerate(mod_files):
        file = str(os.path.join(mod_folder, filename))
        print('Reading file:', file)
        df = pd.read_csv(file,delimiter=',')
    
        # s.body_dict["thorax"] = "LClavicle" 
        # s.body_dict["thorax"] = "MidShoulder"  
        Q, P, P_target, times = [], [], [], []
        s.prob = None
        s.reset()
        for t in range(0,df.shape[0]):
            target = []
            for kp in kps:
                if kp == '':
                    target.append([np.nan,np.nan,np.nan])
                # elif kp == 'LHip':
                #     LASI = np.array(df.loc[t,["Marker.Skeleton:LASI.Position.X","Marker.Skeleton:LASI.Position.Y","Marker.Skeleton:LASI.Position.Z"]].to_numpy()).squeeze()/1000
                #     LPSI = np.array(df.loc[t,["Marker.Skeleton:LPSI.Position.X","Marker.Skeleton:LPSI.Position.Y","Marker.Skeleton:LPSI.Position.Z"]].to_numpy()).squeeze()/1000
                #     target.append((LASI+LPSI)/2)        
                # elif kp == 'RHip':
                #     RASI = np.array(df.loc[t,["Marker.Skeleton:RASI.Position.X","Marker.Skeleton:RASI.Position.Y","Marker.Skeleton:RASI.Position.Z"]].to_numpy()).squeeze()/1000
                #     RPSI = np.array(df.loc[t,["Marker.Skeleton:RPSI.Position.X","Marker.Skeleton:RPSI.Position.Y","Marker.Skeleton:RPSI.Position.Z"]].to_numpy()).squeeze()/1000
                #     target.append((RASI+RPSI)/2) 
                elif kp == 'Midshoulder':
                    RClav = np.array(df.loc[t,["Bone.Skeleton:RShoulder.Position.X","Bone.Skeleton:RShoulder.Position.Y","Bone.Skeleton:RShoulder.Position.Z"]].to_numpy()).squeeze()/1000
                    LClave = np.array(df.loc[t,["Bone.Skeleton:LShoulder.Position.X","Bone.Skeleton:LShoulder.Position.Y","Bone.Skeleton:LShoulder.Position.Z"]].to_numpy()).squeeze()/1000
                    target.append((RClav+LClave) / 2)    # target.append((RCAJ+LCAJ) / 2)
                else:
                    target.append(np.array(df.loc[t,["Bone.Skeleton:"+kp+".Position.X","Bone.Skeleton:"+kp+".Position.Y","Bone.Skeleton:"+kp+".Position.Z"]].to_numpy()).squeeze()/1000)
            
            target = np.array(target)
            
            # # res = np.dot(R,target.reshape(-1,3).transpose()*1000) + T
            # Rx = np.eye(3)
            # Rx = np.array([[1,0,0], [0,0,1], [0,-1,0]])   # Enrico
            Rx = np.array([[0,0,1], [1,0,0], [0,1,0]]).T  # Shafagh

            res = np.dot(Rx,target.reshape(-1,3).transpose()*1000)
            target = res.transpose()/1000  

            # s12.load_from_numpy(target.reshape(-1,3),s.kps)
            # s.load_from_BODY12(s12)  
            s15.load_from_numpy(target.reshape(-1,3),s.kps)
            s.load_from_BODY15(s15) 
            
            # s.estimate_confidence()
            if t == 0:
                s.exact_scale()
            else:
                s.exact_scale(max_iterations=10)    
            # for i,b in enumerate(s.body_dict.keys()):
            #     print(i, b, s.body_dict[b], s._nimble.getBodyScales().reshape(-1,3)[i])
            
            # s.qpIK(1000,0.01)

            gui.nativeAPI().renderSkeleton(s._nimble)
            
            pos = s._nimble.getJointWorldPositions(s.joints)
            pos = s.correct(pos)

            target = target.reshape(-1,1)

            print(kps)
            print(s.joints[1].getName())

            target_color, nimble_color = np.array([0,255,0, 0.8]), np.array([0,0,0, 1])
            for i in range(0,int(target.shape[0]/3)):
                color = np.array([255,0,0, 1]) if i == 1 else target_color
                gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.01,0.01,0.01]),pos=target[3*i:3*i+3],color=color) 
            for i in range(0,int(pos.shape[0]/3)):
                color = np.array([0,0,255, 0.8]) if i == 1 else nimble_color
                gui.nativeAPI().createSphere(key=s.joints[i].getName(), radii=np.array([0.01,0.01,0.01]),pos=pos[3*i:3*i+3],color=color)
            Q.append(s._nimble.getPositions())
            P.append(s.to_numpy())
            P_target.append(target)
            s.reset_history()
            # break
            # time.sleep(0.01)
            # break
        Q = np.array(Q)
        P = np.array(P)
        P_target = np.array(P_target)
        # print(np.mean(times))

        df = pd.DataFrame(data=np.array(Q),columns=header)

        COMETH_folder = '../data/COMETH_angles/' 
        COMETH_filename = filename.replace('.oneheader.csv','.angles_COMETH.csv')
        df.to_csv(str(os.path.join(COMETH_folder, COMETH_filename)))
        print('Writing file:', COMETH_filename)

extract_COMETH_angles(1)

# for subj in range(2, n_subjects-1):
    # modify_csv_files(subj)
    # extract_COMETH_angles(subj)


In [ ]:

# def extract_COMETH_angles(subj):
#     side = healthy_arm[subj]

#     # Read the modified version of the csv in pandas
#     mod_folder = '../data/raw_exports/modified/'
#     mod_files = [file for file in os.listdir(mod_folder) if f'subject{subj}_{side}h' in file and file.endswith('.csv')]
        
#     for i, file in enumerate(mod_files):
#         filename = str(os.path.join(mod_folder, file))
#         if i == 0:
#             df = pd.read_csv(filename,delimiter=',')
#         else:
#             df = pd.concat([df, pd.read_csv(filename,delimiter=',')],axis=0)

#     s.body_dict["thorax"] = "LClavicle"   
#     Q, P, P_target, times = [], [], [], []
#     s.prob = None
#     s.reset()
#     for t in range(0,df.shape[0]):
#         target = []
#         for kp in kps:
#             if kp == '':
#                 target.append([np.nan,np.nan,np.nan])
#             elif kp == 'LHip':
#                 LASI = np.array(df.loc[t,["Marker.Skeleton:LASI.Position.X","Marker.Skeleton:LASI.Position.Y","Marker.Skeleton:LASI.Position.Z"]].to_numpy()).squeeze()/1000
#                 LPSI = np.array(df.loc[t,["Marker.Skeleton:LPSI.Position.X","Marker.Skeleton:LPSI.Position.Y","Marker.Skeleton:LPSI.Position.Z"]].to_numpy()).squeeze()/1000
#                 target.append((LASI+LPSI) / 2)        
#             elif kp == 'RHip':
#                 RASI = np.array(df.loc[t,["Marker.Skeleton:RASI.Position.X","Marker.Skeleton:RASI.Position.Y","Marker.Skeleton:RASI.Position.Z"]].to_numpy()).squeeze()/1000
#                 RPSI = np.array(df.loc[t,["Marker.Skeleton:RPSI.Position.X","Marker.Skeleton:RPSI.Position.Y","Marker.Skeleton:RPSI.Position.Z"]].to_numpy()).squeeze()/1000
#                 target.append((RASI+RPSI) / 2)                        
#             else:
#                 target.append(np.array(df.loc[t,["Bone.Skeleton:"+kp+".Position.X","Bone.Skeleton:"+kp+".Position.Y","Bone.Skeleton:"+kp+".Position.Z"]].to_numpy()).squeeze()/1000)
        
#         target = np.array(target)
        
#         # # res = np.dot(R,target.reshape(-1,3).transpose()*1000) + T
#         Rx = np.array([[1,0,0], [0,0,1], [0,-1,0]])   # Enrico
#         # Rx = np.array([[0,0,1], [1,0,0], [0,1,0]])  # Shafagh
#         # Rx = np.eye(3)
#         res = np.dot(Rx,target.reshape(-1,3).transpose()*1000)
#         target = res.transpose()/1000  

#         s12.load_from_numpy(target.reshape(-1,3),s.kps)
#         s.load_from_BODY12(s12)
        
#         # s.estimate_confidence()
#         if t == 0:
#             s.exact_scale()
#         else:
#             s.exact_scale(max_iterations=10)    

#         # s.qpIK(1000,0.01)
        
#         gui.nativeAPI().renderSkeleton(s._nimble)
        
#         pos = s._nimble.getJointWorldPositions(s.joints)
#         pos = s.correct(pos)

#         target = target.reshape(-1,1)
#         for i in range(0,int(target.shape[0]/3)):
#             gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.01,0.01,0.01]),pos=target[3*i:3*i+3],color=np.array([0,255,0,0.8]))
        
#         for i in range(0,int(pos.shape[0]/3)):
#             gui.nativeAPI().createSphere(key=s.joints[i].getName(), radii=np.array([0.01,0.01,0.01]),pos=pos[3*i:3*i+3],color=np.array([0,0,0,1]))
#         Q.append(s._nimble.getPositions())
#         P.append(s.to_numpy())
#         P_target.append(target)
#         s.reset_history()
#         # break
#         # time.sleep(0.01)
#         # break
#     Q = np.array(Q)
#     P = np.array(P)
#     P_target = np.array(P_target)
#     # print(np.mean(times))

#     df = pd.DataFrame(data=np.array(Q),columns=header)

#     COMETH_folder = '../data/COMETH/' 
#     COMETH_filename = f'subject{subj}_{side}h.angles_COMETH.csv'
#     filename = str(os.path.join(COMETH_folder, COMETH_filename))
#     df.to_csv(filename)


    # for subj in range(n_subjects):
    #     modify_csv_files(subj)
    #     extract_COMETH_angles(subj)